# A Simple Example of a [Data-driven Journalism](https://en.wikipedia.org/wiki/Data-driven_journalism) Application
---

In [1]:
#Pkg.add("CSV")
#Pkg.add("DataFrames")
#Pkg.add("Query")
using CSV
using DataFrames
#using Query

# Question 1

_Which are the people that are affiliated to a Brazilian political party and also are public servants from a Brazilian federal university?_

## Data

###  Collecting

* List of all public servants from the [Brazilian Official Transparency Web portal](http://transparencia.gov.br). 
* List of [all people affiliated to all Brazilian political parties](http://dados.gov.br/dataset/filiados-partidos-politicos) from the [Brazilian Offical Open Data Portal](http://dados.gov.br).

_Remark - BUGFIX: We faced problems when loading the origianl files so we created copied versions of these files to work on._

### Cleaning

In order to load the data sets, we have to perform the following cleaning tasks.

* Convert files from ISO-8859-1 to UFT-8 enconding with `iconv`.
* Remove empty fields because of [`CSV.jl`'s Issue#183](https://github.com/JuliaData/CSV.jl/issues/183) with sed by replacing it with "Sem informacao" string.
* For the federal public servatns data set: replace `BC[0-9]xxx` fields to `0000xxx` so CSV won't complain about converting `B` to `Int641`.
* For the federal public servatns data set: remove the first line of the file with `tail` as `sed` adds an empty line in the begining of the file.

The following Bash commands do all this job. Please remark that the last command will take $\sim20$ secs on an i7 processor with SSD disk.

```bash
cd data-ddj/filiados-al/aplic-7/sead/lista_filiados/uf
iconv -f ISO-8859-1 filiados_pt_al.csv -t UTF-8 | sed 's/\"\"/\"Sem informacao\"/g' > filiados_pt_al-clean.csv
cd -
cd data-ddj/servidores/201806_Servidores/
unzip 20180630-full-dataset.zip
cd 20180630-full-dataset
iconv -f ISO-8859-1 20180630_Cadastro.csv -t UTF-8  | sed 's/\"\"/\"Sem informacao\"/g' | sed 's/\"BC[0-9]/\"0000/g'  | tail -n +2 > 20180630_Cadastro-clean.csv
```

### Loading

In [14]:
# Loading dataset: all people affiliate to the political party (ex.: PT)
path_ptal = "data-ddj/filiados-al/aplic-7/sead/lista_filiados/uf/filiados_pt_al-clean.csv"
ptal = CSV.read(path_ptal,delim=";")

,DATA DA EXTRACAO,HORA DA EXTRACAO,NUMERO DA INSCRICAO,NOME DO FILIADO,SIGLA DO PARTIDO,NOME DO PARTIDO,UF,CODIGO DO MUNICIPIO,NOME DO MUNICIPIO,ZONA ELEITORAL,SECAO ELEITORAL,DATA DA FILIACAO,SITUACAO DO REGISTRO,TIPO DO REGISTRO,DATA DO PROCESSAMENTO,DATA DA DESFILIACAO,DATA DO CANCELAMENTO,DATA DA REGULARIZACAO,MOTIVO DO CANCELAMENTO
1,29/07/2018,00:57:37,24221531783,CARLOS HENRIQUE GUEDES DE ALMEIDA,PT,PARTIDO DOS TRABALHADORES,AL,27855,MACEIÓ,1,39,15/09/2004,REGULAR,OFICIAL,14/04/2018,Sem informacao,Sem informacao,Sem informacao,Sem informacao
2,29/07/2018,00:57:37,26591771740,DJALBA JOAQUIM DOS SANTOS,PT,PARTIDO DOS TRABALHADORES,AL,27197,BOCA DA MATA,48,56,25/09/2007,REGULAR,OFICIAL,15/04/2016,Sem informacao,Sem informacao,Sem informacao,Sem informacao
3,29/07/2018,00:57:37,17819191791,JOSE MATIAS DOS SANTOS,PT,PARTIDO DOS TRABALHADORES,AL,28932,TEOTÔNIO VILELA,34,132,03/07/2003,REGULAR,OFICIAL,14/04/2018,Sem informacao,Sem informacao,Sem informacao,Sem informacao
4,29/07/2018,00:57:37,4356561708,RICARDO JOSE MORONI VALENCA,PT,PARTIDO DOS TRABALHADORES,AL,27855,MACEIÓ,2,199,24/12/1987,CANCELADO,OFICIAL,14/04/2018,Sem informacao,02/03/2016,Sem informacao,Judicial
5,29/07/2018,00:57:37,2965641708,PEDRO CIRILO DOS SANTOS,PT,PARTIDO DOS TRABALHADORES,AL,28436,POÇO DAS TRINCHEIRAS,50,18,08/07/1999,REGULAR,OFICIAL,16/04/2015,Sem informacao,Sem informacao,Sem informacao,Sem informacao
6,29/07/2018,00:57:37,24600801767,REGIVANIO PINTO,PT,PARTIDO DOS TRABALHADORES,AL,28436,POÇO DAS TRINCHEIRAS,50,32,29/07/1999,REGULAR,OFICIAL,16/04/2015,Sem informacao,Sem informacao,Sem informacao,Sem informacao
7,29/07/2018,00:57:37,16963271708,MARINALVA DE OMENA SILVA,PT,PARTIDO DOS TRABALHADORES,AL,27855,MACEIÓ,2,93,02/04/2007,REGULAR,OFICIAL,15/04/2016,Sem informacao,Sem informacao,Sem informacao,Sem informacao
8,29/07/2018,00:57:37,25753411708,GEDIVAN COSTA,PT,PARTIDO DOS TRABALHADORES,AL,27855,MACEIÓ,3,56,02/09/2007,REGULAR,OFICIAL,15/04/2016,Sem informacao,Sem informacao,Sem informacao,Sem informacao
9,29/07/2018,00:57:37,18522211716,JOSILENE DA SILVA SANTOS,PT,PARTIDO DOS TRABALHADORES,AL,27855,MACEIÓ,3,37,07/07/1999,REGULAR,OFICIAL,15/04/2016,Sem informacao,Sem informacao,Sem informacao,Sem informacao
10,29/07/2018,00:57:37,31821091708,ASSIS ALEXANDRE DA SILVA,PT,PARTIDO DOS TRABALHADORES,AL,27278,CAMPO ALEGRE,47,55,10/05/2004,REGULAR,OFICIAL,16/10/2015,Sem informacao,Sem informacao,Sem informacao,Sem informacao


In [15]:
# Loading dataset: all federal public servants (ex.: only active public servants)
tic()
path_pservants = "data-ddj/servidores/201806_Servidores/20180630-full-dataset/20180630_Cadastro-clean.csv"
#TODO path_pservant_afastados = ...
pservants = CSV.read(path_pservants,delim=";")
#TODO pservant_afastados = CSV.read(path_pservant_afastados,delim=";")
toc()

elapsed time: 37.758236406 seconds


37.758236406

## Filter 

* Getting all active ("regular" status) people from the party.
* Getting the names of all public servants from UFAL and are affiliate to the party.

In [16]:
regulares_ptal = []
for row in eachrow(ptal)
    if row[Symbol("SITUACAO DO REGISTRO")] == "REGULAR"
        push!(regulares_ptal,row[Symbol("NOME DO FILIADO")])
    end
end

In [17]:
pservants_ufal_pt = []
for row in eachrow(pservants) #TODO include pservant_afastados
    if row[Symbol("ORG_EXERCICIO")] == "UNIVERSIDADE FEDERAL DE ALAGOAS"
        #push!(pservants_ufal_pt,row[Symbol("NOME")])
        #TODO Handle homonymous people, e.g., by using Brazilian CPFs (social security numbers)
        if findfirst(regulares_ptal,row[Symbol("NOME")]) != 0
            push!(pservants_ufal_pt,row[Symbol("NOME")])
        end
    end
end

# TODO Example using Query.jl
# query = @from i in pservants_ufal begin
#     @where i.Symbol("NOME") == 
# end

## Visualize

**DISCLAIMER**: Owing to homonymous people and to the data sets which might not be updated, the results showed here might not reflect the reality.

In [23]:
pservants_ufal_pt

c = ""
for p in pservants_ufal_pt
    c = "$(c) - $(p)"
end
write("a",c)

2164

## Story

TODO were affiliated to TODO party in TODO and also is a public servant at TODO because she/he believes that TODO.

---
# Question 2
_How are distributed the political party affiliation in a Brazilian federal university?_

---
# Question 3
_How often people change political parties?_

---
# Question 4
_How often people join and/or quit a political party?_

---
# Question 5
_Which are the reasons for people quitting the parties? Which is the party that has more judicial reasons for quitting the party?_
